In [ ]:
%matplotlib inline
from ipyparallel import Client, error
cluster=Client(profile="mpi")
view=cluster[:]
view.block=True

try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

```{note}
This feature requires MPI, and may not be able to be run on Colab or Binder.
```

# Parallel Groups

When systems are added to a `ParallelGroup`, they will be executed in parallel, assuming that the `ParallelGroup` is given an MPI communicator of sufficient size.  Adding subsystems to a ParallelGroup is no different than adding them to a normal Group.  For example:

In [ ]:
%%px

import openmdao.api as om

prob = om.Problem()
model = prob.model

model.set_input_defaults('x', 1.)

parallel = model.add_subsystem('parallel', om.ParallelGroup(), 
                               promotes_inputs=[('c1.x', 'x'), ('c2.x', 'x'), 
                                                ('c3.x', 'x'), ('c4.x', 'x')])
parallel.add_subsystem('c1', om.ExecComp(['y=-2.0*x']))
parallel.add_subsystem('c2', om.ExecComp(['y=5.0*x']))
parallel.add_subsystem('c3', om.ExecComp(['y=-3.0*x']))
parallel.add_subsystem('c4', om.ExecComp(['y=4.0*x']))

model.add_subsystem('c5', om.ExecComp(['y=3.0*x1 + 7.0*x2 - 2.0*x3 + x4']))

model.connect("parallel.c1.y", "c5.x1")
model.connect("parallel.c2.y", "c5.x2")
model.connect("parallel.c3.y", "c5.x3")
model.connect("parallel.c4.y", "c5.x4")

prob.setup(check=False, mode='fwd')
prob.run_model()

print(prob['c5.y'])

In [ ]:
%%px

from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob['c5.y'], 39.0, 1e-6)

In this example, components *c1* through *c4* will be executed in parallel, provided that the `ParallelGroup` is given four MPI processes.  If the name of the python file containing our example were `my_par_model.py`, we could run it under MPI and give it two processes using the following command:

```
    mpirun -n 4 python my_par_model.py
```


```{Note}
This will only work if you've installed the mpi4py and petsc4py python packages, which are not installed by default in OpenMDAO.
```


In the previous example, all four components in the `ParallelGroup` required just a single MPI process, but
what happens if we want to add subsystems to a `ParallelGroup` that has other processor requirements?
In OpenMDAO, we control process allocation behavior by setting the `min_procs` and/or `max_procs` or
`proc_weight` args when we call the `add_subsystem` function to add a particular subsystem to
a `ParallelGroup`.

```{eval-rst}
    .. automethod:: openmdao.core.group.Group.add_subsystem
        :noindex:
```


If you use both `min_procs/max_procs` and `proc_weight`, it can become less obvious what the
resulting process allocation will be, so you may want to stick to just using one or the other.
The algorithm used for the allocation starts, assuming that the number of processes is greater than or
equal to the number of subsystems, by assigning the `min_procs` for each subsystem.  It then adds
any remaining processes to subsystems based on their weights, being careful not to exceed their
specified `max_procs`, if any.

If the number of processes is less than the number of subsystems, then each subsystem, one at a
time, starting with the one with the highest `proc_weight`, is allocated to the least-loaded process.
An exception will be raised if any of the subsystems in this case have a `min_procs` value greater than one.